In [1]:
# example from https://subscription.packtpub.com/book/data/9781800200937/2/ch02lvl1sec09/bi-directional-lstms-bilstms
# !pip install tensorflow_datasets
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

# datasets in different domains such as images, audio, video, text, summarization, and so on.
", ".join(tfds.list_builders())

2023-02-07 17:14:26.330359: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


'abstract_reasoning, accentdb, aeslc, aflw2k3d, ag_news_subset, ai2_arc, ai2_arc_with_ir, amazon_us_reviews, anli, answer_equivalence, arc, asqa, asset, assin2, bair_robot_pushing_small, bccd, beans, bee_dataset, beir, big_patent, bigearthnet, billsum, binarized_mnist, binary_alpha_digits, ble_wind_field, blimp, booksum, bool_q, bucc, c4, c4_wsrs, caltech101, caltech_birds2010, caltech_birds2011, cardiotox, cars196, cassava, cats_vs_dogs, celeb_a, celeb_a_hq, cfq, cherry_blossoms, chexpert, cifar10, cifar100, cifar100_n, cifar10_1, cifar10_corrupted, cifar10_n, citrus_leaves, cityscapes, civil_comments, clevr, clic, clinc_oos, cmaterdb, cnn_dailymail, coco, coco_captions, coil100, colorectal_histology, colorectal_histology_large, common_voice, conll2002, conll2003, controlled_noisy_web_labels, coqa, cos_e, cosmos_qa, covid19, covid19sum, crema_d, criteo, cs_restaurants, curated_breast_imaging_ddsm, cycle_gan, d4rl_adroit_door, d4rl_adroit_hammer, d4rl_adroit_pen, d4rl_adroit_relocate, 

In [2]:
imdb_train, ds_info = tfds.load(name="imdb_reviews", split="train",
                                with_info=True, as_supervised=True)
imdb_test = tfds.load(name="imdb_reviews", split="test",
                      as_supervised=True)

print(ds_info)



Metal device set to: Apple M1
tfds.core.DatasetInfo(
    name='imdb_reviews',
    full_name='imdb_reviews/plain_text/1.0.0',
    description="""
    Large Movie Review Dataset. This is a dataset for binary sentiment
    classification containing substantially more data than previous benchmark
    datasets. We provide a set of 25,000 highly polar movie reviews for training,
    and 25,000 for testing. There is additional unlabeled data for use as well.
    """,
    config_description="""
    Plain text
    """,
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    data_path='/Users/kkepins-macwro_1/tensorflow_datasets/imdb_reviews/plain_text/1.0.0',
    file_format=tfrecord,
    download_size=80.23 MiB,
    dataset_size=129.83 MiB,
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
        'text': Text(shape=(), dtype=string),
    }),
    supervised_keys=('text', 'label'),
    disable_shuffling=False,
    splits={
        'test': <

2023-02-07 17:14:32.962976: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-07 17:14:32.963794: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
for example, label in imdb_train.take(1):
    print(example, '\n', label)

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string) 
 tf.Tensor(0, shape=(), dtype=int64)


2023-02-07 17:14:36.110198: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-07 17:14:36.151701: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [4]:
# Normalization and vectorization

tokenizer = tfds.deprecated.text.Tokenizer()
vocabulary_set = set()
MAX_TOKENS = 0
for example, label in imdb_train:
    some_tokens = tokenizer.tokenize(example.numpy())
    if MAX_TOKENS < len(some_tokens):
        MAX_TOKENS = len(some_tokens)
    vocabulary_set.update(some_tokens)

imdb_encoder = tfds.deprecated.text.TokenTextEncoder(vocabulary_set, tokenizer=tokenizer)
vocab_size = imdb_encoder.vocab_size
print(vocab_size, MAX_TOKENS)

93931 2525


In [5]:
for example, label in imdb_train.take(1):
    print(example)
    encoded = imdb_encoder.encode(example.numpy())
    print(imdb_encoder.decode(encoded))

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)
This was an absolutely terrible movie Don t be lured in by Christopher Walken or Michael Ironside Both are great actors but this must simply be their worst role in history Even their great acting could not redeem this movie s ridiculous storyline This 

In [6]:
imdb_encoder.save_to_file("imdb_reviews_vocab")

enc = tfds.deprecated.text.TokenTextEncoder.load_from_file("imdb_reviews_vocab")
enc.decode(enc.encode("Good case. Excellent value."))

'Good case Excellent value'

In [8]:
# Tokenization: Reviews need to be tokenized into words.
# Encoding: These words need to be mapped to integers using the vocabulary.
# Padding: Reviews can have variable lengths, but LSTMs expect vectors of the same length. So, a constant length is chosen. Reviews shorter than this length are padded with a specific vocabulary index, usually 0 in TensorFlow. Reviews longer than this length are truncated. Fortunately, TensorFlow provides such a function out of the box.
# Padding is an important step as different layers in TensorFlow cannot handle tensors of different widths. Tensors of different widths are called ragged tensors. There is ongoing work to incorporate support for ragged tensors and the support is improving. However, the support for ragged tensors is not universal in TensorFlow. Consequently, ragged tensors are avoided in this text.
#
def encode_pad_transform(sample):
    encoded = imdb_encoder.encode(sample.numpy())
    pad = tf.keras.preprocessing.sequence.pad_sequences([encoded], padding='post',
                                 maxlen=150)
    return np.array(pad[0], dtype=np.int64)

def encode_tf_fn(sample, label):
    encoded = tf.py_function(encode_pad_transform,
                             inp=[sample],
                             Tout=(tf.int64))
    encoded.set_shape([None])
    label.set_shape([])
    return encoded, label

subset = imdb_train.take(10)
tst = subset.map(encode_tf_fn)
for review, label in tst.take(1):
    print(review, label)
    print(imdb_encoder.decode(review))



tf.Tensor(
[24828 52509 91146 33056 76933 43743 89760 17462 40811   157 43654 18681
 64533 30881 45538 70976 29743 46232 37012 80432   109 64064 39837 47785
 76726 40811 76153 19654 48328 43654 84908 12953 76153 80432 65773 92389
 69454 84734 39837 43743 68337 48754 75445 24828 43743 28313 91146 12918
  9229  8799 26990 34201 18917 29600 19443 13767 42369 88431 80360 87848
 41606 47793 42369 60067 76153 63873 38753 85413 40248 68733 60041 57835
 91054 26081 41091 82365 84520 20591 19253 30881 52509 89193 64064 51726
 19443 74004 53542 43654 51726 43743 34912 52509 36574 27524 77984 81633
 36281 30426 78686 19260 34912 33659 37012 63558 26101 39837 39840 31573
 68337 26101 64533 30881 68337 24656 28819 30426 92389 32968  3090 16611
 37068     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0], shape=(150,), dtype=int64) tf.Tensor(0, shape=(), dtype=int64)
This was an

In [12]:
# Passing the additional parameter enables TensorFlow to use multiple subprocesses to execute the transformation on.
encoded_train = imdb_train.map(encode_tf_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)
encoded_test = imdb_test.map(encode_tf_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)

vocab_size = imdb_encoder.vocab_size
# The embedding dimension
embedding_dim = 64
# Number of RNN units
rnn_units = 64
# batch size
BATCH_SIZE=100

encoded_train

<ParallelMapDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [21]:
# LSTM

import time

def build_model_lstm(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model


start = time.time()

lstm = build_model_lstm(
    vocab_size = vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)
# model.summary()

lstm.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'Precision', 'Recall'])
encoded_train_batched = encoded_train.batch(BATCH_SIZE)
lstm.fit(encoded_train_batched, epochs=5)

lstm.evaluate(encoded_test.batch(BATCH_SIZE))

print("--- %s seconds ---" % (time.time() - start))


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (100, None, 64)           6011584   
                                                                 
 lstm_6 (LSTM)               (100, 64)                 33024     
                                                                 
 dense_6 (Dense)             (100, 1)                  65        
                                                                 
Total params: 6,044,673
Trainable params: 6,044,673
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5


2023-02-06 18:16:03.552865: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-06 18:16:03.930593: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-06 18:16:04.572778: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


250/250 [==============================] - 58s 217ms/step - loss: 0.4511 - accuracy: 0.7718 - precision: 0.7647 - recall: 0.7853
Epoch 2/5
250/250 [==============================] - 52s 209ms/step - loss: 0.1981 - accuracy: 0.9264 - precision: 0.9252 - recall: 0.9278
Epoch 3/5
250/250 [==============================] - 52s 206ms/step - loss: 0.1542 - accuracy: 0.9515 - precision: 0.9532 - recall: 0.9497
Epoch 4/5
250/250 [==============================] - 48s 193ms/step - loss: 0.1327 - accuracy: 0.9592 - precision: 0.9585 - recall: 0.9601
Epoch 5/5
250/250 [==============================] - 53s 211ms/step - loss: 0.0788 - accuracy: 0.9776 - precision: 0.9771 - recall: 0.9781


2023-02-06 18:20:24.612019: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-06 18:20:24.875329: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


250/250 [==============================] - 35s 136ms/step - loss: 0.7079 - accuracy: 0.8347 - precision: 0.8037 - recall: 0.8856
--- 298.0240819454193 seconds ---


In [23]:
# BiLSTM model

start = time.time()

def build_model_bilstm(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  mask_zero=True,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(rnn_units)),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

bilstm = build_model_bilstm(
    vocab_size = vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)
# bilstm.summary()

bilstm.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'Precision', 'Recall'])
encoded_train_batched = encoded_train.batch(BATCH_SIZE)

bilstm.fit(encoded_train_batched, epochs=5)

bilstm.evaluate(encoded_test.batch(BATCH_SIZE))

print("--- %s seconds ---" % (time.time() - start))


2023-02-06 18:21:23.711660: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-06 18:21:24.340516: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-06 18:21:24.457537: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-06 18:21:25.561455: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-06 18:21:26.057922: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


250/250 [==============================] - 98s 351ms/step - loss: 0.4961 - accuracy: 0.7458 - precision: 0.7369 - recall: 0.7646
Epoch 2/5
250/250 [==============================] - 83s 331ms/step - loss: 0.2268 - accuracy: 0.9171 - precision: 0.9218 - recall: 0.9116
Epoch 3/5
250/250 [==============================] - 78s 308ms/step - loss: 0.1349 - accuracy: 0.9593 - precision: 0.9603 - recall: 0.9582
Epoch 4/5
250/250 [==============================] - 89s 354ms/step - loss: 0.1027 - accuracy: 0.9693 - precision: 0.9704 - recall: 0.9682
Epoch 5/5
250/250 [==============================] - 81s 326ms/step - loss: 0.0723 - accuracy: 0.9814 - precision: 0.9801 - recall: 0.9828


2023-02-06 18:28:28.847520: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-06 18:28:29.142422: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-06 18:28:29.215667: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


250/250 [==============================] - 46s 174ms/step - loss: 0.7695 - accuracy: 0.8192 - precision: 0.7826 - recall: 0.8838
--- 476.38784313201904 seconds ---
Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (100, None, 64)           6011584   
                                                                 
 bidirectional_4 (Bidirectio  (100, 128)               66048     
 nal)                                                            
                                                                 
 dense_8 (Dense)             (100, 1)                  129       
                                                                 
Total params: 6,077,761
Trainable params: 6,077,761
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5


2023-02-06 18:29:16.739129: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-06 18:29:17.317620: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-06 18:29:17.422944: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-06 18:29:18.443949: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-06 18:29:18.534564: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


250/250 [==============================] - 78s 289ms/step - loss: 0.4522 - accuracy: 0.7702 - precision: 0.7606 - recall: 0.7886
Epoch 2/5
250/250 [==============================] - 70s 280ms/step - loss: 0.1914 - accuracy: 0.9290 - precision: 0.9280 - recall: 0.9302
Epoch 3/5
250/250 [==============================] - 70s 280ms/step - loss: 0.1161 - accuracy: 0.9651 - precision: 0.9650 - recall: 0.9653
Epoch 4/5
250/250 [==============================] - 103s 412ms/step - loss: 0.1033 - accuracy: 0.9682 - precision: 0.9697 - recall: 0.9667
Epoch 5/5
250/250 [==============================] - 82s 329ms/step - loss: 0.0841 - accuracy: 0.9768 - precision: 0.9769 - recall: 0.9766


2023-02-06 18:35:58.487153: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-06 18:35:58.802715: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-06 18:35:58.883676: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


250/250 [==============================] - 44s 165ms/step - loss: 0.5229 - accuracy: 0.8208 - precision: 0.7669 - recall: 0.9220
--- 447.8993580341339 seconds ---


In [13]:

import time


start = time.time()

# text_vectorizer = tf.keras.layers.TextVectorization(
#     max_tokens=n_vocab, standardize=None,
#     split=None, input_shape=(window_size,)
# )
# Train the model on existing data
# text_vectorizer.adapt(train_ds)
gru_model = tf.keras.models.Sequential([
    # text_vectorizer,
    tf.keras.layers.Embedding(vocab_size+2, 96),
    tf.keras.layers.GRU(512, return_sequences=True),
    tf.keras.layers.GRU(256, return_sequences=True),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(vocab_size, activation='softmax')
])


gru_model.compile(loss='binary_crossentropy',
               optimizer='adam',
               metrics=['accuracy', 'Precision', 'Recall'])
encoded_train_batched = encoded_train.batch(BATCH_SIZE)

gru_model.fit(encoded_train_batched, epochs=5)

gru_model.evaluate(encoded_test.batch(BATCH_SIZE))

print("--- %s seconds ---" % (time.time() - start))


Epoch 1/5


ValueError: in user code:

    File "/Users/kkepins-macwro_1/.local/share/virtualenvs/data-science-upskills-eseJr18D/lib/python3.10/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/Users/kkepins-macwro_1/.local/share/virtualenvs/data-science-upskills-eseJr18D/lib/python3.10/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/kkepins-macwro_1/.local/share/virtualenvs/data-science-upskills-eseJr18D/lib/python3.10/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/Users/kkepins-macwro_1/.local/share/virtualenvs/data-science-upskills-eseJr18D/lib/python3.10/site-packages/keras/engine/training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/kkepins-macwro_1/.local/share/virtual